In [2]:
import pandas as pd

# CSV 파일 읽기
ts2000_df = pd.read_csv('ts2000_winsorized.csv')
delisted_df = pd.read_csv('상장폐지기업_v2.csv')

# 종목코드를 6자리 문자열로 변환하는 함수
def format_stock_code(code):
    """종목코드를 6자리 문자열로 변환 (앞자리 0으로 채움)"""
    return str(code).zfill(6)

# ts2000_df의 stock_code를 6자리 문자열로 변환
ts2000_df['stock_code_formatted'] = ts2000_df['stock_code'].apply(format_stock_code)

# 상장폐지 기업의 종목코드를 6자리 문자열로 변환
delisted_df['종목코드_formatted'] = delisted_df['종목코드'].apply(format_stock_code)

# 상장폐지 종목코드 세트 생성 (빠른 검색을 위해)
delisted_codes = set(delisted_df['종목코드_formatted'])

# 먼저 모든 기업을 0으로 초기화
ts2000_df['is_defaulted'] = 0

# 상장폐지 기업들에 대해서만 처리
delisted_companies = ts2000_df[ts2000_df['stock_code_formatted'].isin(delisted_codes)]

# 각 상장폐지 기업의 가장 마지막 연도 찾기
for stock_code in delisted_codes:
    company_data = ts2000_df[ts2000_df['stock_code_formatted'] == stock_code]
    if not company_data.empty:
        # 해당 기업의 가장 마지막 연도 찾기
        max_year = company_data['year'].max()
        # 해당 기업의 마지막 연도 데이터만 is_defaulted = 1로 설정
        mask = (ts2000_df['stock_code_formatted'] == stock_code) & (ts2000_df['year'] == max_year)
        ts2000_df.loc[mask, 'is_defaulted'] = 1

# 임시로 생성한 formatted 컬럼 제거 (필요에 따라 유지 가능)
ts2000_df = ts2000_df.drop('stock_code_formatted', axis=1)

# 결과 확인
print("상장폐지 기업 매칭 결과:")
print(f"전체 기업 수: {len(ts2000_df)}")
print(f"상장폐지 기업 수: {ts2000_df['is_defaulted'].sum()}")
print(f"정상 기업 수: {(ts2000_df['is_defaulted'] == 0).sum()}")

# 상장폐지 기업 목록 확인 (샘플)
defaulted_companies = ts2000_df[ts2000_df['is_defaulted'] == 1][['corp_nm', 'stock_code', 'year']].drop_duplicates()
print(f"\n상장폐지 기업 예시 (상위 10개):")
print(defaulted_companies.head(10))

# 결과를 새 파일로 저장
ts2000_df.to_csv('ts2000_with_default_flag.csv', index=False, encoding='utf-8-sig')
print(f"\n결과가 'ts2000_with_default_flag.csv' 파일로 저장되었습니다.")

상장폐지 기업 매칭 결과:
전체 기업 수: 21833
상장폐지 기업 수: 92
정상 기업 수: 21741

상장폐지 기업 예시 (상위 10개):
        corp_nm  stock_code  year
389       (주)금빛       45890  2020
702      (주)넥솔론      110570  2016
705      (주)넥스지       81970  2017
951      (주)뉴로스      126870  2021
1196  (주)대성합동지주        5620  2016
1468    (주)데코앤이       17680  2018
1609    (주)동성화학        5190  2020
2526  (주)매직마이크로      127160  2020
2541    (주)맥스로텍      141070  2020
2693      (주)모다      149940  2019

결과가 'ts2000_with_default_flag.csv' 파일로 저장되었습니다.
